# Medical image retrieval
Let's focus on building the absolut bare minimum to have something working. With medical focus we should def be able to impress some people.

Proposal: Implement a super simple version of Retrieval-augmented Biomedical Visual Question Answering https://arxiv.org/abs/2303.00534

Use case: Doctor uploads a 2D medical image and gets a text description.

Technical approach (veeeery basic, Notebook-only):
1. Get pairs of medical image + report
2. Create image embedding via DINOv2
3. Store image embedding in vector database (index, img_embed, text)
4. When doctor provides new image, create new image embedding, select 1 nearest neighbour from vector db, return the corresponding text

This would give us a very tiny core, hopefully working. Then, we grow it out, think: UI via streamlit, leverage text embeddings, better retrieval augmentation (e.g., 3 nearest neighbours -> summarize respective 3 text), include truelens for verification, etc


TODO

- [ ] Store 1 example image into vector DB by adding padding
- [ ] Build streamlit UI
- [ ] Get basic dataset of 50 images + report
- [ ]

## Setup

### Install dependencies
Let's install some of the dependencies for this notebook if we don't have them already

In [ ]:
! pip install trulens-eval==0.15.1 llama_index>=0.8.29post1 pinecone-client>=2.2.2 nltk>=3.8.1 html2text>=2020.1.16

Reason for being yanked: Unstable release


### Add API keys
For this quickstart, you will need Open AI and Huggingface keys

In [84]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["PINECONE_API_KEY"] = ""
os.environ["PINECONE_ENVIRONMENT"] = "gcp-starter"

### Import from LlamaIndex and TruLens

In [ ]:
import pinecone
import torch
import numpy as np
import torchvision.transforms as T
from PIL import Image, UnidentifiedImageError
import os
from tqdm import tqdm
from io import BytesIO
from urllib.parse import urlparse
import json
import requests
import cv2
import time
import hashlib
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index import (
    VectorStoreIndex,
    SimpleWebPageReader,
    LLMPredictor,
    ServiceContext
)

from trulens_eval import TruLlama, Feedback, Tru, feedback
from trulens_eval.feedback import GroundTruthAgreement, Groundedness
from pathlib import Path

tru = Tru()


/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


### First we need to load documents. We can use SimpleWebPageReader

Next we can create the vector store in pinecone.

In [ ]:
index_name = "medical-images"

PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_ENVIRONMENT = os.getenv('PINECONE_ENVIRONMENT')

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT
)

# create the index
pinecone.create_index(
        name=index_name,
        dimension=512
    )

# set vector store as pinecone
vector_store = PineconeVectorStore(
        index_name=index_name,
        environment=os.environ["PINECONE_ENVIRONMENT"]
    )

index = pinecone.Index(index_name)
index

ApiException: ignored

# Download crawled iamges

In [ ]:
def read_json_line_by_line(file_path):
    parsed_data = []
    with open(file_path, 'r') as f:
        for line in f:
            parsed_line = json.loads(line.strip())
            parsed_data.append(parsed_line)
    return parsed_data

def download_image(url):
  image_name = os.path.basename(urlparse(url).path)
  response = requests.get(url, stream=True)

  if response.status_code == 200:
    # Create the full file path
    full_path = os.path.join('/content/img', image_name)

    # Write the contents of the response to a file
    with open(full_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)

  elif response.status_code == 429:  # Wait and try again when Too many requests...
    time.sleep(10)
    download_image(url)
  else:
      print(f"Unable to download image. HTTP response code: {response.status_code}")

file_path = '/content/result_page_1-3.json'
raw_data = read_json_line_by_line(file_path)

for image_case in tqdm(raw_data, desc="Downloading images"):
  url = image_case["image_download_url"]
  download_image(url)

Unable to download image. HTTP response code: 403


Unable to download image. HTTP response code: 403


Unable to download image. HTTP response code: 403


Unable to download image. HTTP response code: 403


# AI

In [ ]:
index = pinecone.Index(index_name)
index

In [ ]:
dinov2_vits14 = torch.hub.load("facebookresearch/dinov2", "dinov2_vits14")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dinov2_vits14.to(device)

transform_image = T.Compose([T.ToTensor(),
                             T.Resize(224),
                             T.CenterCrop(224),
                             T.Normalize([0.5], [0.5])])

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vits14_pretrain.pth
100%|██████████| 84.2M/84.2M [00:00<00:00, 168MB/s]


In [80]:
def compute_embedding(img) -> dict:
    """
    Create an index that contains all of the images in the specified list of files.
    """
    with torch.no_grad():
            embedding = dinov2_vits14(load_image(img).to(device))
            embeddings_numpy = np.array(embedding[0].cpu().numpy()).reshape(1, -1)
            padded_embedding = pad_embedding(embeddings_numpy)

    return padded_embedding

def pad_embedding(embedding: np.ndarray, target_dim: int = 512) -> np.ndarray:
    """
    Pad the given embedding with zeros to match the target dimension.
    """
    original_dim = embedding.shape[1]
    padding_dim = target_dim - original_dim

    if padding_dim > 0:
        padding = np.zeros((1, padding_dim))
        padded_embedding = np.hstack([embedding, padding])
    else:
        padded_embedding = embedding

    return padded_embedding


def img_to_vector_db(img, report, download_path):
  embedding = compute_embedding(img)
  unique_id = hash_file(download_path)
  single_vector = {
      'id': unique_id,
      'values': embedding.flatten().tolist(),
      'metadata': {'report': report, 'download_path': download_path}
  }
  upsert_response = index.upsert(vectors=[single_vector])
  print(f"Inserted {unique_id} to {index}")

def hash_file(image_path: str) -> str:
    """
    Hash the filename to create a unique ID.
    """
    filename = image_path.split("/")[-1]
    unique_id = hashlib.sha256(filename.encode()).hexdigest()[0:6]
    return unique_id

def read_json_line_by_line(file_path):
    parsed_data = []
    with open(file_path, 'r') as f:
        for line in f:
            parsed_line = json.loads(line.strip())
            parsed_data.append(parsed_line)
    return parsed_data

def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    return img

def load_image(image: Image) -> torch.Tensor:
  """
  Load a DICOM dataset and return a tensor that can be used as an input to DINOv2.
  """
  return transform_image(image)[:3].unsqueeze(0)


def create_report(raw_data: dict):
  return raw_data['title'] + '. ' + ' '.join(line['text'] for line in raw_data['finding_lines'])

In [81]:
file_path = '/content/result_page_1-14.json'
raw_data = read_json_line_by_line(file_path)

In [82]:
for image_case in tqdm(raw_data, desc="Processing images"):
    try:
        report = create_report(image_case)
        download_path = image_case["image_download_url"]
        image = download_image(download_path)
        img_to_vector_db(img=image, report=report, download_path=download_path)
    except UnidentifiedImageError:
        print(f"Skipping faulty image from URL: {download_path}")
        continue

Processing images:   0%|          | 1/1278 [00:00<14:54,  1.43it/s]

Inserted d150d0 to <pinecone.index.Index object at 0x7ef6fd0335e0>


Processing images:   0%|          | 2/1278 [00:01<11:53,  1.79it/s]

Inserted 63fe47 to <pinecone.index.Index object at 0x7ef6fd0335e0>


Processing images:   0%|          | 3/1278 [00:02<15:34,  1.36it/s]

...

Inserted b9956c to <pinecone.index.Index object at 0x7ef6fd0335e0>


Processing images:  57%|█████▋    | 727/1278 [07:25<04:38,  1.98it/s]

Inserted b9956c to <pinecone.index.Index object at 0x7ef6fd0335e0>


Processing images:  57%|█████▋    | 727/1278 [07:26<05:38,  1.63it/s]


OSError: ignored

In [ ]:
def predict(img_path):
  embedding = compute_embedding(img_path)
  response = index.query(
    vector=embedding.flatten().tolist(),
    top_k=3,
    include_values=True,
    include_metadata=True
  )
  return response


In [ ]:
img_path = "/content/Chest Xray Dataset.png"
new_img = Image.open(img_path).convert("RGB")

result = predict(img_path=img_path)
print(f"ID: {result['matches'][0]['id']} | Similarity score: {round(result['matches'][0]['score'], 2)}")
print(f"Report: {result['matches'][0]['metadata']['report']}")
new_img


In [ ]:
result

{'matches': [{'id': 'b52904',
              'metadata': {'report': 'Rheumatic valvular heart disease. Gross '
                                     'cardiomegaly with increased subcarinal '
                                     'angle and double cardiac border, '
                                     'suggesting left atrium enlargement.\n'
                                     'Bilateral CP angles are blunted? Mild '
                                     'bilateral pleural effusion.'},
              'score': 0.911326349,
              'values': [0.642509699,
                         2.62609768,
                         -2.58366227,
                         4.78221798,
                         -4.28256798,
                         0.0297071114,
                         0.167398125,
                         0.558124065,
                         2.96692276,
                         0.281901181,
                         0.594820917,
                         3.64164424,
                         